# Introdução

## 📘 Sobre este Estudo

Este notebook foi elaborado com fins **didáticos e demonstrativos** para mostrar como aplicar técnicas de **inferência estatística**, utilizando tanto métodos **clássicos (paramétricos)** quanto **computacionais** (como bootstrap e testes de aleatorização).

---

## 🛒 Contexto do Problema

A empresa fictícia **RetailX**, uma varejista digital, lançou uma campanha para testar o impacto de diferentes estratégias promocionais sobre o comportamento de seus clientes. A empresa está especialmente interessada em entender:

- Se as campanhas aumentam o engajamento no site
- Se as campanhas influenciam a taxa de conversão (compra)
- Se há impacto no valor total gasto

Para isso, um experimento foi conduzido com **quatro grupos de clientes**, atribuídos aleatoriamente.

---

## 🎯 Objetivo do Estudo

Avaliar se **diferentes tipos de campanha** geram **diferenças estatisticamente significativas** em:
- Taxa de conversão (realizou compra ou não)
- Valor de compra (gasto em reais)
- Número de visitas ao site (antes e depois da campanha)
---

## 🧾 Descrição das Variáveis

| Variável            | Tipo        | Descrição                                                                 |
|---------------------|-------------|---------------------------------------------------------------------------|
| `grupo`             | Categórica  | **Campanha recebida pelo cliente**:<br>• `A`: grupo **controle** (não recebeu estímulo)<br>• `B`: cupom de desconto<br>• `C`: frete grátis<br>• `D`: cashback |
| `evento`            | Binária     | Indicador de conversão: o cliente **realizou uma compra?** (`1` = sim, `0` = não) |
| `score`             | Contínua    | Valor total gasto pelo cliente (em reais) durante o período da campanha  |
| `pre_treinamento`   | Contínua    | Número de visitas ao site **antes da campanha**                           |
| `pos_treinamento`   | Contínua    | Número de visitas ao site **após a campanha**                             |

---

## 🧠 O que significa “grupo controle”?

O grupo **controle** (`grupo = A`) é formado por clientes que **não receberam nenhuma campanha promocional**. Esse grupo serve como **referência neutra**, permitindo que possamos comparar com os demais grupos e **isolar o efeito da campanha**.

> Exemplo: Se o grupo B (cupom) tiver maior conversão do que o grupo A (controle), é possível afirmar com maior confiança que o cupom teve impacto real — desde que a diferença seja estatisticamente significativa.

# Análises

In [23]:
# SETUP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, t, f

In [24]:
# LOAD: A base já está sanitizada.
base = pd.read_excel("dados/base_retailx.xlsx")
base.head()

,grupo,evento,score,pre_treinamento,pos_treinamento
0,A,0,0.00,6,6
1,A,1,70.68,4,4
2,A,1,60.23,6,6
3,A,1,102.16,8,8
4,A,0,0.00,1,2


## 1. Construa um intervalo de confiança de 95% para estimar a proporção de clientes que realizaram compra.

In [25]:
# Variável observada
X = base["evento"].values

# Tamanho da amostra
n = X.shape[0]

# Proporção amostral observada
p_obs = X.mean()

print(f"n = {n}")
print(f"p_obs = {np.round(p_obs, 2)}")

n = 300
p_obs = 0.57


### Modelo Paramétrico

<img src="img/e1a.jpeg" width="500">
<img src="img/e1b.jpeg" width="500">

In [26]:
# Melhor estimativa para p
p = p_obs

# Validação de aproximação pela normal
print((n*p >= 10) & (n*(1-p) >= 10))

True


In [27]:
# Erro Padrão
SE = np.sqrt(p*(1-p)/n)
print(f"SE = {np.round(SE, 2)}")

SE = 0.03


In [28]:
# Obtenção do Zc
gama = .95
q    = (1- gama)/2 + gama
Zc   = norm.ppf(q=q)
print(f"Zc = {np.round(Zc, 2)}")

Zc = 1.96


In [29]:
# Intervalo de confiança
p0 = p_obs - Zc*SE
p1 = p_obs + Zc*SE
print(f"IC = [{np.round(p0, 2)}, {np.round(p1, 2)}]")
print(f"IC = {np.round(p_obs, 2)} +- {np.round(Zc, 2)} * {np.round(SE, 2)}")

IC = [0.51, 0.62]
IC = 0.57 +- 1.96 * 0.03


**Conclusão**

- Temos **95% de confiança** de que o verdadeiro valor da proporção populacional \(p\) está entre **51% e 62%**. Em outras palavras, acreditamos com 95% de confiança que a **taxa de conversão dos clientes dessa loja** se encontra nesse intervalo.

- Essa confiança **não deve ser interpretada** como a probabilidade de que este intervalo específico contenha \(p\), pois o valor de \(p\) é fixo (embora desconhecido), e o intervalo é aleatório.

- A interpretação correta é: Se repetíssemos esse mesmo processo de amostragem e construção de intervalo muitas vezes, em **95% dos casos** os intervalos obtidos conteriam o verdadeiro valor de \(p\).

- Portanto, com base nessa confiança no processo, podemos selecionar **este único intervalo** e afirmar com **95% de confiança** que o verdadeiro valor de \(p\) está entre **51% e 62%**.


## 2. Podemos afirmar que a taxa de conversão dos clientes é superior que 50% ?

**Hipóteses do Teste**

- **H₀ (Hipótese nula):** A taxa de conversão na população **não é superior a 50%**. O valor observado pode ser explicado apenas pela variabilidade amostral (acaso na seleção da amostra).
- **H₁ (Hipótese alternativa):** A taxa de conversão na população **é superior a 50%**.

**Notação formal:**

- H₀: p = po | (po = 50%)
- H₁: p > po



### Modelo Paramétrico

<img src="img/e2.jpeg" width="500">

In [30]:
# Nível de significância do teste
alpha = .05
print(f"alpha = {np.round(alpha, 2)}")

alpha = 0.05


In [31]:
# Distribuição sob H0
po = .5
print(f"p0 = {np.round(po, 2)}")

p0 = 0.5


In [32]:
# Padronizando o valor observado
Z_obs = (p_obs - po)/np.sqrt(po*(1-po)/n)
print(f"Z_obs = {np.round(Z_obs, 2)}")

Z_obs = 2.31


In [33]:
# Calcular valor-p para teste unicaudal à direita (Ha: p > p0)
p_value = 1 - norm.cdf(Z_obs)

# Decisão
if p_value < alpha:
    print(f"p-valor = {p_value:.2f} < α = {alpha} → Rejeitamos H0")
else:
    print(f"p-valor = {p_value:.2f} ≥ α = {alpha} → Não rejeitamos H0")


p-valor = 0.01 < α = 0.05 → Rejeitamos H0


**Conclusão**

- Ao nível de significância de 5% (probabilidade de rejeitar incorretamente a hipótese nula), temos evidência suficiente para rejeitar H₀ em favor de H₁.  
Com isso, concluímos que a taxa de conversão dos clientes é **superior a 50%**.
